In [ ]:
# pip install keybert
# pip install keybert transformers
# !pip install kiwipiepy

In [29]:
import pandas as pd
from keybert import KeyBERT
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel

# 데이터 Load & Preprocess

In [13]:
df=pd.read_csv('sample_split_sentences.csv',encoding='utf-8-sig')

In [14]:
df.head()

,Unnamed: 0,rst_name,review_text,review_sentence_split
0,0,가장맛있는족발 구의역점,잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서...,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...
1,1,강나루 유황오리주물럭 본점,생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ 늘 갈때마다 맛있어요 ㅎㅎ 가...,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ..."
2,2,고고기,금요일 저녁이라 고기가 땡겨서 고고기로 왔어요 고 기는 다 좋아하지만 두툼하고 질 ...,"['금요일 저녁이라 고기가 땡겨서 고고기로 왔어요', '고 기는 다 좋아하지만 두툼..."
3,3,고공 구의점,구의역 소고기 맛집입니다 토시살 갈비살 버 섯구이 구성의 소고기 세트 주문...,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세..."
4,4,고기로,고기로 돼지반판 주문했습니다 구의동 오션키즈 맞 은편이예요 옛 황금돼지 터 ...,"['고기로 돼지반판 주문했습니다', '구의동 오션키즈 맞 은편이예요', '옛 황금..."


In [6]:
df=df.drop(columns=['Unnamed: 0','review_text'])

In [ ]:
"""문자열 형태의 리스트를 파이썬 리스트로 변환하는 작업. 필요하면 사용"""
import ast  # 문자열 형태의 리스트를 파이썬 리스트로 변환하기 위해 필요

# 문자열 형태의 리스트를 파이썬 리스트로 변환
df["review_sentence_split"] = df["review_sentence_split"].apply(ast.literal_eval)

# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
df["review_sentence_split"].iloc[0]

def text_join(row):
    text=row.review_sentence_split
    result=" ".join(text)
    return result

# df['joined_text']=df.apply(text_join,axis=1)
# df.drop(columns=['review_sentence_split'])

# nli-mean-tokens로 사전학습된 모델 가져옴

In [21]:
# 불필요하게 많은 공백은 처리해줌
document=df['review_text'].iloc[0]
document=document.replace('  ',' ')
document

'잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 좋네요  매운 거 좋아  . 아이들이랑 가서 반반 족발 먹었어요 지점마다 서비스 내용이 다른가 봅니다 일단 콩나물국 정말 맛있었어요 서비스 5개 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다 반찬들이 정갈하고 하나같이 맛이 좋아요 족발은 냄새 하나도 안나고 부드럽습니다 아이들은 기본 족발을 먹고 저는 매운 족발을 상추와 알배추에 싸 먹었어요 정말 맛있게 잘 먹었습니다 . 양도 푸짐하고 곁들여 먹을 수 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 좋아요. 구의동 먹자골목에서 롱런하는 이유가 있죠 맨날 배달로만 먹다가 매장에서 먹으니 더 꿀맛입니다 윤기가 좔좔 흐르는거 보이시죠 체인이라고 다 같은 체인은 아니잖아요 친절한 사장님 덕에 맛있고 편안하게 잘 먹고왔어요  매장 안 여기저기서 반반족발을 드시던데 진짜 둘 다 맛있어서 놓치고 싶지않아요 한톨도 남기지 않고 순삭하고는 터지는 배를 부여잡고 나왔네요 ㅎㅎ. 퇴근하고 현지인은 약속이라도 한듯 이 집으로 집결한다 왜이리 적게 받으시지 싶을정도로 음식에 진심인 집 오랫동안 텃세높은 광진구에서 살아남은 비결이 고스란히 느껴지는 집 1 보쌈 버너 먹는 동안 따뜻하고 촉촉한 고기를 맛 볼 수 있다 2 김치 보쌈 집의 근본인 김치가 무려 10종 김치를 먹기 위해 보쌈을 시킬 정도 3 서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 중 취향껏 선택 4 푸짐한 양 둘이가서 푸파하려다가 실패하긴 처음 . 광진구족발원탑답게 대존맛임  역시 족발은 가족   넘넘 맛있게 먹었어요 서비스로 선택한 얼큰순두부찌개도 대존맛. 이름그대로 가장 맛있는 족발 보쌈 구의최고맛집입니다 믿고 드시면됨 서비스도 많음. 포장하러 갔는데 남자 사장님께서 너무 친절하시고 좋았어요  메뉴도 푸짐하고 양도 많고 전화로도 배민처럼 리뷰 이벤트 가능했는데 받고 오자마자 

In [23]:
kw_model = KeyBERT(model="distilbert-base-nli-mean-tokens")
keywords = kw_model.extract_keywords(document)
print(keywords)

[('광진구족발원탑답게', 0.9225), ('반반족발시켜', 0.8908), ('불친절합니다', 0.8865), ('잘먹구갑니당', 0.8823), ('날치알주먹밥을', 0.8792)]


# kiwi 사용 pos 태깅

In [ ]:
document

In [26]:
kiwi = Kiwi()
kiwi.analyze(document)

[([Token(form='잡내', tag='NNG', start=0, len=2),
   Token(form='안', tag='MAG', start=3, len=1),
   Token(form='나', tag='VV', start=4, len=1),
   Token(form='고', tag='EC', start=5, len=1),
   Token(form='맛있', tag='VA', start=7, len=2),
   Token(form='어요', tag='EC', start=9, len=2),
   Token(form='ᆼ', tag='Z_CODA', start=10, len=1),
   Token(form='구의', tag='NNP', start=12, len=2),
   Token(form='먹자골목', tag='NNG', start=15, len=4),
   Token(form='바로', tag='MAG', start=20, len=2),
   Token(form='앞', tag='NNG', start=23, len=1),
   Token(form='에서', tag='JKB', start=24, len=2),
   Token(form='접', tag='NNG', start=27, len=1),
   Token(form='근성', tag='NNG', start=29, len=2),
   Token(form='이', tag='JKS', start=31, len=1),
   Token(form='매우', tag='MAG', start=33, len=2),
   Token(form='좋', tag='VA', start=36, len=1),
   Token(form='습니다', tag='EC', start=37, len=3),
   Token(form='ᆼ', tag='Z_CODA', start=39, len=1),
   Token(form='지나가', tag='VV', start=41, len=3),
   Token(form='면서', tag='EC', st

# 특정 품사 구만 추출 하는 함수, 목적에 따라 바꿈

In [149]:
# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith("N") or pos.startswith("SL")):
            results.append(token)
    return results

#### 부사,조사,어미 제거 함수

In [224]:
# 명사 추출 함수
def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results

In [100]:
not "JX".startswith("J")

False

In [ ]:
# 명사 추출 함수
def extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if (
            len(token) != 1
            and pos.startswith("N")
            or pos.startswith("SL")
            or pos.startswith("V")
        ):
            results.append(token)
    return results

In [30]:
model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [158]:
def extract_keywords_from_reviews(document, top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1, 1),  # 단어 n-gram 범위
        stop_words=None,  # 불용어
        use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        highlight=True
    )  # 상위 n개 키워드
    print(f"Keywords: {keywords}\n")
    print("-" * 80)  # 리뷰 사이 구분선

#### 현재는 명사,동사,형용사 구만 추출했는데, 전체 리뷰를 쓰되 부사구만 제거 해볼 예정

In [33]:
for i in range(3):
    #식당의 문장 리스트 추출
    review_doc = df["review_text"].iloc[i]
    review_noun_tagging=noun_extractor(review_doc)
    review_noun_tagging=" ".join(review_noun_tagging)

    print(f"Restaurant : {df['rst_name'].iloc[i]}")
    extract_keywords_from_reviews(review_noun_tagging)

Restaurant : 가장맛있는족발 구의역점
Keywords: [('좋아하', 0.7322), ('생각나', 0.7156), ('곁들이', 0.7109), ('지나가', 0.7086), ('맛나', 0.6955), ('쫄깃하', 0.6931), ('현지인', 0.6931), ('돋우', 0.6915), ('아이', 0.6876), ('밑반찬', 0.6802), ('쫀득하', 0.6794), ('정갈하', 0.679), ('원하', 0.671), ('드시', 0.6681), ('쏘맥', 0.6586), ('시키', 0.6576), ('말씀', 0.6545), ('문의', 0.6497), ('즐겁', 0.649), ('깨끗하', 0.63)]

--------------------------------------------------------------------------------
Restaurant : 강나루 유황오리주물럭 본점
Keywords: [('맛있', 0.6869), ('맛나', 0.646), ('배고프', 0.6431), ('반찬', 0.6221), ('맞이', 0.6221), ('훌륭하', 0.6164), ('한잔', 0.6161), ('어쩌', 0.6086), ('비하', 0.608), ('오리구', 0.606), ('식사', 0.6053), ('좋아하', 0.6046), ('편하', 0.603), ('비린내', 0.6029), ('왔습니', 0.5943), ('볶음밥', 0.5937), ('맛집', 0.5906), ('아저씨', 0.5884), ('챙기', 0.5809), ('더워요', 0.5788)]

--------------------------------------------------------------------------------
Restaurant : 고고기
Keywords: [('대존잘', 0.5886), ('셔보세요', 0.5837), ('못하', 0.5824), ('맛잏어요', 0.578), ('빵빵하', 0.5774

# 메뉴를 담고있는 리뷰만 추출해서 진행해봄

In [82]:
test = [
    "역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  .",
    "근처 순대국중 괜춘   .",
    "구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯.",
    "구의동 순대국 원탑맛집.",
    "오소리 국밥  순대국밥  야채순대 먹었어요",
    "2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   .",
    "고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요",
    "순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요",
    "와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ.",
    "어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당.",
    "순대국  고기모듬  술국 등 모든 메뉴가 맛있어요",
    "순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요",
    "순대국밥 이랑 막걸리 ㅋㅋ.",
    "순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃.",
    "순대국이 맛있어요.",
    "맛집이라 순대국 국물이 진해요",
    "순대국 깔끔.",
    "맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ.",
    "맛있는 순대국집  가격이 올라 별하나 뺍니다.",
    "순대국밥등 넘 맛있어요",
]

In [83]:
test1=" ".join(test)
test1

'역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  . 근처 순대국중 괜춘   . 구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯. 구의동 순대국 원탑맛집. 오소리 국밥  순대국밥  야채순대 먹었어요 2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   . 고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요 순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요 와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ. 어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당. 순대국  고기모듬  술국 등 모든 메뉴가 맛있어요 순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요 순대국밥 이랑 막걸리 ㅋㅋ. 순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃. 순대국이 맛있어요. 맛집이라 순대국 국물이 진해요 순대국 깔끔. 맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ. 맛있는 순대국집  가격이 올라 별하나 뺍니다. 순대국밥등 넘 맛있어요'

In [98]:
extract_keywords_from_reviews(test1)

Keywords: [('순대국밥을', 0.7202), ('순대국밥등', 0.6857), ('맛집이라', 0.6679), ('순대국밥', 0.641), ('끓여주셨어요', 0.6182), ('주문하면', 0.6155), ('잘먹었습니다', 0.6127), ('신경써주시면', 0.6055), ('적셔봅니당', 0.5961), ('원탑맛집', 0.5941), ('순대국중에', 0.5869), ('해줘서', 0.5843), ('맛은', 0.5792), ('데워달라니', 0.5782), ('시키면', 0.5751), ('괜춘', 0.5731), ('맛있고', 0.5728), ('맛집', 0.5698), ('부속고기들도', 0.5037), ('포장', 0.4673)]

--------------------------------------------------------------------------------


### pos태깅 후 목적에 맞게 특정 구 제거

In [108]:
test2=noun_extractor(test1)

In [109]:
test2

['순대국',
 '먹거리',
 '맛있',
 '둔하',
 '근처',
 '순대국',
 '구의역',
 '근처',
 '순대국',
 '일등',
 '맛집',
 '사장',
 '친절',
 '아쉽',
 '배민',
 '안함',
 '머릿고기',
 '순대국',
 '포장',
 '맛있',
 '위생',
 '신경',
 '구의동',
 '순대국',
 '맛집',
 '오소리',
 '국밥',
 '순대국',
 '야채',
 '순대',
 '포장',
 '주문',
 '대부분',
 '인분',
 '포장',
 '포장',
 '국물',
 '인분',
 '건더기',
 '인분',
 '포장',
 '고기',
 '모듬',
 '순대',
 '수육',
 '부속',
 '고기',
 '순대국',
 '맛있',
 '소주',
 '구의',
 '순대국',
 '어르신',
 '사이',
 '막걸리',
 '순대국',
 '순대국',
 '고기',
 '모듬',
 '술국',
 '메뉴',
 '맛있',
 '순대국',
 '장국',
 '일대',
 '맛나',
 '중독',
 '순대국',
 '막걸리',
 '순대국',
 '얼큰하',
 '진하',
 '맛있',
 '모둠',
 '고기',
 '부드럽',
 '순대국',
 '맛있',
 '맛집',
 '순대국',
 '국물',
 '진하',
 '순대국',
 '깔끔',
 '모둠',
 '중양',
 '아쉽',
 '맛있',
 '순대국',
 '가격',
 '하나',
 '순대',
 '국밥',
 '맛있']

In [110]:
" ".join(test2)

'순대국 먹거리 맛있 둔하 근처 순대국 구의역 근처 순대국 일등 맛집 사장 친절 아쉽 배민 안함 머릿고기 순대국 포장 맛있 위생 신경 구의동 순대국 맛집 오소리 국밥 순대국 야채 순대 포장 주문 대부분 인분 포장 포장 국물 인분 건더기 인분 포장 고기 모듬 순대 수육 부속 고기 순대국 맛있 소주 구의 순대국 어르신 사이 막걸리 순대국 순대국 고기 모듬 술국 메뉴 맛있 순대국 장국 일대 맛나 중독 순대국 막걸리 순대국 얼큰하 진하 맛있 모둠 고기 부드럽 순대국 맛있 맛집 순대국 국물 진하 순대국 깔끔 모둠 중양 아쉽 맛있 순대국 가격 하나 순대 국밥 맛있'

In [111]:
extract_keywords_from_reviews(" ".join(test2))

Keywords: [('맛나', 0.5996), ('진하', 0.5803), ('술국', 0.5796), ('건더기', 0.578), ('구의역', 0.5679), ('둔하', 0.5554), ('구의', 0.5479), ('근처', 0.5394), ('안함', 0.5315), ('막걸리', 0.5254), ('오소리', 0.4898), ('인분', 0.4797), ('주문', 0.4594), ('순대', 0.4512), ('소주', 0.4288), ('머릿고기', 0.4101), ('하나', 0.4076), ('일대', 0.4056), ('얼큰하', 0.3954), ('순대국', 0.3943)]

--------------------------------------------------------------------------------


# 부사구 제거 후 추출

In [101]:
test1

'역시 순대국  먹거리집 역시 맛있어요  둔둔하게 잘먹었습니다  . 근처 순대국중 괜춘   . 구의역 근처 순대국 일등 맛집  여사장님 친절하시고 아쉽게 이제 배민은 안함 머릿고기  순대국 포장 여전히 맛있고 푸짐한데 위생은 조금 더 신경써주시면 좋을 듯. 구의동 순대국 원탑맛집. 오소리 국밥  순대국밥  야채순대 먹었어요 2인분 포장 주문하면 대부분 각 1인분씩 포장 해줘서 따로 말 안 했는데 2인분을 합포장 국물2인분 건더기 2인분  해줘서 1인분씩 포장 아니냐니까 말 안   . 고기모듬 시키면 순대도 수육도 부속고기들도 다 나와서 좋아요 순대국도 맛있고 소주 마시다 식어서 데워달라니 다시 바글바글 끓여주셨어요 와   구의에서 먹어본 순대국중에 1등이네요 ㅎㅎㅎ. 어르신들 사이에 껴서 저도 막걸리에 순대국밥을 적셔봅니당. 순대국  고기모듬  술국 등 모든 메뉴가 맛있어요 순대국 내장국 이일대에서 제일 맛나요   중독돼서 자주 가고 있어요 순대국밥 이랑 막걸리 ㅋㅋ. 순대국도 얼큰하니 진하고 맛있구   모둠고기도 부드럽고 쫄깃쫄깃. 순대국이 맛있어요. 맛집이라 순대국 국물이 진해요 순대국 깔끔. 맛은 있는데 모둠중양이너무 적어서 아쉬웠어요ㅜㅜ. 맛있는 순대국집  가격이 올라 별하나 뺍니다. 순대국밥등 넘 맛있어요'

In [102]:
test3=adverb_remover(test1)
test3

['순대국',
 '먹거리',
 '맛있',
 '둔하',
 '근처',
 '순대국',
 '구의역',
 '근처',
 '순대국',
 '일등',
 '맛집',
 '사장',
 '친절',
 '아쉽',
 '배민',
 '안함',
 '머릿고기',
 '순대국',
 '포장',
 '맛있',
 '푸짐',
 '위생',
 '신경',
 '구의동',
 '순대국',
 '맛집',
 '오소리',
 '국밥',
 '순대국',
 '야채',
 '순대',
 '포장',
 '주문',
 '대부분',
 '인분',
 '포장',
 '포장',
 '국물',
 '인분',
 '건더기',
 '인분',
 '포장',
 '아니',
 '이',
 '고기',
 '모듬',
 '시키',
 '순대',
 '수육',
 '부속',
 '고기',
 '나오',
 '순대국',
 '맛있',
 '소주',
 '마시',
 '데우',
 '끓이',
 '구의',
 '순대국',
 'ㅎㅎㅎ',
 '어르신',
 '사이',
 '막걸리',
 '순대국',
 '적시',
 '순대국',
 '고기',
 '모듬',
 '술국',
 '모든',
 '메뉴',
 '맛있',
 '순대국',
 '장국',
 '일대',
 '맛나',
 '중독',
 '순대국',
 '막걸리',
 'ㅋㅋ',
 '순대국',
 '얼큰하',
 '진하',
 '맛있',
 '모둠',
 '고기',
 '부드럽',
 '순대국',
 '맛있',
 '맛집',
 '순대국',
 '국물',
 '진하',
 '순대국',
 '깔끔',
 '모둠',
 '중양',
 '아쉽',
 'ㅜㅜ',
 '맛있',
 '순대국',
 '가격',
 '하나',
 '순대',
 '국밥',
 '맛있']

In [103]:
tmp=kiwi.analyze(" ".join(test3))

In [104]:
tmp

[([Token(form='순대국', tag='NNG', start=0, len=3),
   Token(form='먹거리', tag='NNG', start=4, len=3),
   Token(form='맛', tag='NNG', start=8, len=1),
   Token(form='이', tag='JKS', start=9, len=1),
   Token(form='ᆻ', tag='Z_CODA', start=9, len=1),
   Token(form='둔하', tag='VA', start=11, len=2),
   Token(form='근처', tag='NNG', start=14, len=2),
   Token(form='순대국', tag='NNG', start=17, len=3),
   Token(form='구의역', tag='NNP', start=21, len=3),
   Token(form='근처', tag='NNG', start=25, len=2),
   Token(form='순대국', tag='NNG', start=28, len=3),
   Token(form='일등', tag='NNG', start=32, len=2),
   Token(form='맛집', tag='NNG', start=35, len=2),
   Token(form='사장', tag='NNG', start=38, len=2),
   Token(form='친절', tag='NNG', start=41, len=2),
   Token(form='아쉽', tag='VA-I', start=44, len=2),
   Token(form='배미', tag='NNG', start=47, len=2),
   Token(form='ᆫ', tag='JX', start=48, len=1),
   Token(form='안', tag='MAG', start=50, len=1),
   Token(form='하', tag='VV', start=51, len=1),
   Token(form='ᆷ', tag='E

In [89]:
for t, p,s,l in tmp[0][0]:
    if p.startswith("J"):
        print(f"으아아아아아앙아아아아앙아아아악!!!!!비상!!!!!{p}")
    print(f"t: {t}, p= {p}, s={s}, l={l}")

t: 순대국, p= NNG, s=0, l=3
t: 먹거리, p= NNG, s=3, l=3
t: 맛있, p= VA, s=6, l=2
t: 둔하, p= VA, s=8, l=2
t: 근처, p= NNG, s=10, l=2
t: 순대국, p= NNG, s=12, l=3
t: 구의역, p= NNP, s=15, l=3
t: 근처, p= NNG, s=18, l=2
t: 순대국, p= NNG, s=20, l=3
t: 일등, p= NNG, s=23, l=2
t: 맛집, p= NNG, s=25, l=2
t: 사장, p= NNG, s=27, l=2
t: 친절, p= NNG, s=29, l=2
t: 아쉽, p= VA-I, s=31, l=2
t: 배미, p= NNG, s=33, l=2
으아아아아아앙아아아아앙아아아악!!!!!비상!!!!!JX
t: ᆫ, p= JX, s=34, l=1
t: 안, p= MAG, s=35, l=1
t: 하, p= VV, s=36, l=1
t: ᆷ, p= EF, s=36, l=1
t: 머릿고기, p= NNG, s=37, l=4
t: 순대국, p= NNG, s=41, l=3
t: 포장, p= NNG, s=44, l=2
t: 맛있, p= VA, s=46, l=2
t: 푸짐, p= XR, s=48, l=2
t: 위생, p= NNG, s=50, l=2
t: 신경, p= NNG, s=52, l=2
t: 구의동, p= NNP, s=54, l=3
t: 순대국, p= NNG, s=57, l=3
t: 맛집, p= NNG, s=60, l=2
t: 오소리, p= NNG, s=62, l=3
t: 국밥, p= NNG, s=65, l=2
t: 순대국, p= NNG, s=67, l=3
t: 야채, p= NNG, s=70, l=2
t: 순대, p= NNG, s=72, l=2
t: 포장, p= NNG, s=74, l=2
t: 주문, p= NNG, s=76, l=2
t: 대부분, p= NNG, s=78, l=3
t: 인분, p= NNG, s=81, l=2
t: 포장, p= NNG, s=83,

In [105]:
test3=" ".join(test3)

In [106]:
extract_keywords_from_reviews(test3)

Keywords: [('ㅎㅎㅎ', 0.7061), ('맛나', 0.6228), ('건더기', 0.621), ('술국', 0.6113), ('진하', 0.6091), ('구의역', 0.6032), ('ㅋㅋ', 0.6025), ('둔하', 0.5927), ('시키', 0.5884), ('구의', 0.5792), ('근처', 0.5778), ('데우', 0.5774), ('안함', 0.574), ('막걸리', 0.5626), ('ㅜㅜ', 0.5498), ('오소리', 0.5268), ('인분', 0.5197), ('주문', 0.4941), ('순대', 0.4845), ('머릿고기', 0.4534)]

--------------------------------------------------------------------------------


# 메뉴 추출한 샘플들로 돌려보자!

In [207]:
data=pd.read_csv('menu_token_setnece_sample.csv')
data=data.drop(columns=['Unnamed: 0'])

KeyError: "['Unnamed: 0'] not found in axis"

In [208]:
data.head(5)

,rst_name,review_sentence_split,menu_name_split,menu_sentence
0,가장맛있는족발 구의역점,['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...,"['', '왕족', '알뜰', ')', '불족발', '실속', '보쌈', '싱글족발...",['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가...
1,강나루 유황오리주물럭 본점,"['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '늘 갈때마다 맛있어요 ...","['탕', '양념', '양념주물럭', '오리주물럭', '오리', '주물럭', '마리...","['생 양념 둘다 강추 ㅎㅎ 가면 무조건 반반이쥬 ㅎㅎ', '미나리랑 같이 먹는 오..."
2,고공 구의점,"['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세...","['', '돼지세트', '쫄면', '목살', '고공', '김치찌개', '갈비살', ...","['구의역 소고기 맛집입니다', '토시살 갈비살 버 섯구이 구성의 소고기 세..."
3,고기반햄반김치찌개&김치찜 아차산본점,"['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ...","['4인', '김치찌개', '불고기', '1', '반', '간장제육', '제육', ...","['보글보글 김치찌개에 두부 추가하고 라면사리는 필수 로 넣어먹어요', '김치찌개 ..."
4,고향집,"['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '맛있어요', '새조개 진...","['회', '구이', '새꼬막', '가자미', '참꼬막양념', '생태찌개', '꼬막...","['새조개무침 벌교꼬막 대구탕 새조개무침 미쳤음', '새조개 진짜 비싼데 양..."


In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   rst_name               40 non-null     object
 1   review_sentence_split  40 non-null     object
 2   menu_name_split        40 non-null     object
 3   menu_sentence          40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [209]:
import ast

# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_sentence"] = data["menu_sentence"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
data["menu_sentence"].iloc[0]

['잡내 안나고 맛있어용 구의 먹자골목 바로 앞에서 접 근성이 매우 좋습니당 지나가면서 자주 봤던 곳인데 서비스 메뉴도 알차고 좋네요',
 '매운 거 좋아   .',
 '아이들이랑 가서 반반 족발 먹었어요',
 '지점마다 서비스 내용이 다른가 봅니다',
 '일단 콩나물국 정말 맛있었어요',
 '서비스 5개 중에 막국수와 주먹밥 중에 엄청 고민했는데 아이들 위해서 주먹밥으로 결정했습니다',
 '반찬들이 정갈하고 하나같이 맛이 좋아요',
 '족발은 냄새 하나도 안나고 부드럽습니다',
 '아이들은 기본 족발을 먹고 저는 매운 족발을 상추와 알배추에 싸 먹었어요',
 '정말 맛있게 잘 먹었습니다  .',
 '양도 푸짐하고  곁들여 먹을 수 있는 각종 김치랑 따뜻한 국물에 서비스 메뉴까지 나와서 좋아요.',
 '구의동 먹자',
 '골목에서 롱런하는 이유가 있죠',
 '맨날 배달로만 먹다가 매장에서 먹으니 더 꿀맛입니다  윤기가 좔좔 흐르는거 보이시죠',
 '체인이라고 다 같은 체인은 아니잖아요',
 '친절한 사장님 덕에 맛있고 편안하게 잘 먹고왔어요',
 '매장 안 여기저기서 반반족발을 드시던데 진짜 둘 다 맛있어서 놓치고 싶지않아요',
 '한톨도 남기지 않고 순삭하고는 터지는 배를 부여잡고 나왔네요 ㅎㅎ.',
 '퇴근하고 현지인은 약속이라도 한듯 이 집으로 집결한다',
 '왜이리 적게 받으시지 싶을정도로 음식에 진심인 집  오랫동안 텃세높은 광진구에서 살아남은 비결이 고스란히 느껴지는 집  1  보쌈 버너 먹는 동안 따뜻하고 촉촉한 고기를 맛 볼 수 있다',
 '2  김치 보쌈 집의 근본인 김치가 무려 10종  김치를 먹기 위해 보쌈을 시킬 정도  3  서비스 따뜻한 콩나물국으로 입맛 돋우고 생굴 된찌 순두부찌개 막국수 주먹밥 중 취향껏 선택 4  푸짐한 양 둘이가서 푸파하려다가 실패하긴 처음 .',
 '광진구족발원탑답게 대존맛임',
 '역시 족발은 가족     넘넘 맛있게 먹었어요',
 '서비스로 선택한 얼큰순두부찌개도 대존맛.',
 '이름그대로 가장 맛있는 족발 

In [210]:
# 문자열 형태의 리스트를 파이썬 리스트로 변환
data["menu_name_split"] = data["menu_name_split"].apply(ast.literal_eval)
# 변환된 데이터의 첫 번째 요소 확인을 통해 변환 성공 여부 확인
data["menu_name_split"].iloc[0]

['',
 '왕족',
 '알뜰',
 ')',
 '불족발',
 '실속',
 '보쌈',
 '싱글족발',
 '1',
 '발',
 '싱글보쌈',
 '어리굴젓',
 '통영',
 '불',
 '인',
 '하',
 'g',
 '왕족발',
 '야채',
 '새우젓',
 '알',
 '1인',
 '마늘',
 '수제',
 '싱글불족발',
 '실속알뜰보쌈',
 '족발하고',
 '쌈야채',
 '양념',
 '막국수)',
 '200',
 '알배추',
 '고',
 '족발',
 '수제양념막국수',
 '싱글',
 '200g',
 '쌈장',
 '막국수',
 '겉절이',
 '배추',
 '쌈']

In [218]:
def extract_keywords_from_reviews_candy(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,2),  # 단어 n-gram 범위
        stop_words=None,  # 불용어
        use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드
    print(f"Keywords: {keywords}\n")

In [225]:
for i in range(5):
    # 식당의 문장 리스트 추출
    review_doc = data["menu_sentence"].iloc[i]
    input_text = ".".join(review_doc)
    menu_candidates=data['menu_name_split'].iloc[i]
    input_text=adverb_remover(input_text)
    input_text=" ".join(input_text)
    extract_keywords_from_reviews_candy(input_text,menu_candidates)
    print(f"Restaurant : {data['rst_name'].iloc[i]}")
    print(f"Menu list :{menu_candidates} ")
    print("-" * 80)  # 리뷰 사이 구분선

잡내 맛있 구의 먹자골목 바로 근성 매우 지나가 자주 서비스 메뉴 알차 아이 반반 족발 지점 서비스 내용 다르 일단 
콩나물국 정말 맛있 서비스 막국수 주먹밥 엄청 고민 아이 위하 주먹밥 결정 반찬 정갈 하나같이 족발 냄새 하나 부드럽 
아이 기본 족발 상추 배추 정말 맛있 푸짐 곁들이 각종 김치 따뜻 국물 서비스 메뉴 나오 구의동 골목 이유 맨날 배달 매장
꿀맛 윤기 좔좔 흐르 보이 체인 체인 아니 친절 사장 맛있 편안 여기저기 반반 족발 진짜 맛있 놓치 남기 순삭 터지 부여잡
나오 ㅎㅎ 퇴근 현지인 약속 집결 이리 정도 음식 진심 오랫동안 텃세 광진구 살아남 비결 고스란히 느끼 보쌈 버너 동안 
따뜻하 촉촉 고기 김치 보쌈 근본 김치 무려 10 김치 위하 보쌈 시키 정도 서비스 따뜻 콩나물국 입맛 돋우 된찌 
순두부찌개 막국수 주먹밥 취향 선택 푸짐 푸파 실패 처음 광진 구족 발원 대조 역시 족발 가족 넘넘 맛있 서비스 선택 
얼큰 순두부찌개 대조 이름 그대로 가장 맛있 족발 보쌈 최고 맛집 서비스 포장 남자 사장 너무 친절 메뉴도 푸짐하 전화 
배민 리뷰 이벤트 가능 끓이 사진 순두부 빠지 맛있 오늘 행복 사랑 사람 맛나 족발 반주 족발 쫀득하 너무 맛있 강추 드리
족발 가족 우와 족발 맛있 밑반찬 맛있 가성비 자주 오늘날 족발 쏘맥 즐겁 저녁 족발 너무 맛있 밑반찬 구성 진짜 알차 
서비스 막국수 정말 맛있 자주 방문 처음 처음 방문 맛있 대화 기분 대화 다시 족발 부드럽 맛있 부모 함께 맛있 드시 친절
서비스 아주 친구 맛있 여기 지점 확실히 맛있 부들부들 쫄깃하 진짜 맛있 사이드 메뉴 맛있 가족 끼리 친구 너무 여기 
너무 맛있 상추 진짜 깨끗하 족발 맛있 막국수 후루룩 족발 이번 보쌈 인생 보쌈 ㅜㅜㅜ 진짜 맛있 반찬 너무 나오 구의역 
족발 퇴근 동료 족발 음식 맛있 사장 너무 친절 아주 서비스 고기 맛있 기본 나오 반찬 맛있 음식 빠르 나오 종류 각자 
원하 시키 기본 서비스 하나 무조건 나오 너무 역시 족발 구의동 가족 답십리 족발 생각나 항상 여기 사장 직원 친절 
말모말모 너무 맛있 가족 지점 여기 제일 덕분 데이트 남편 좋아하 지점 냄새 부들 쫄깃 맛있 족발 세계 느끼 우아 서비스 
사장 완전 팍팍 얼마나 살살 매장 따뜻 강추 강추 방문 친절 깔끔 여러 가지 반찬 곁들이 나오 족발 생각나 항상 고기 푸짐
두말 자주 메인 족발 사이드 막국수 맛있 주먹밥 시키 맛나 음식 너무 맛있 깔끔 사람 회식 장소 주문 빠르 빨리 나오 우선
너무 맛있 가족 따끈따끈 맛있 서비스 주먹밥 서비스 맛있 맛나 족발 맛있 체인점 거의 유사 가장 족발 달콤 직원 대부분 
친절 사장 친절 너무 그거 진짜 감사 족발 너무 맛있 피하 드리 자리 시끄럽 아니 오늘 반반 다른 체인점 맛있 가게 곰팡이
냄새 족발 야들야들 맛있 ㅎㅎ ㅎㅎ 여기 족발 시키 막국수 서비스 막국수 엄청 맛있 ㅎㅎ 밑반찬 맛있 ㅎㅎ 사장 친절 
남자 직원 굉장히 친절 본인 전화 조용 불쾌 상황 조심 반반 별루 맛있 쫄깃쫄깃 족발 막국수 정갈하 음식 가족 배부르 
날치 주먹밥 서비스 품질 족발 탱탱 막국수 맛있 맛있 강하 족발 맛나 부추 무침 완전 맛있 테이블 친절 반찬 셀프 직접 
가지 음식 족발 맛있 사진 ㅠㅠ 만두 ㅠㅠ 육수 맛있 배달 맛있 맛있 아이 서비스 자주 가게 포장 맛있 포장 오늘 살코기 
적당 맛있 정말 친절 음식 나오 순서 먼저 순서 아니 둔감 만족 스럽 즐기 막국수 맛있 군대 본인 알바 족발 음식 맛있 
족발 맛잇 가게 자체 냄새 ㅠㅠ 쿰쿰 냄새 가게 전체 ㅠㅠ 알바 친절 반반 족발 시키 국수 서비시 맛나 족발 맛있 만족 
족발 항상 시키 포장 자주 오늘 점원 착각 족발 대자 단골 이벤트 날치 주먹밥 선택 추가 41 000 포장 할인 000 38 000 41 
000 포장 할인 적용 000 추가 이상 말씀 드리 그냥 39 000 계산 나오 다시 체크 착각 아니 세팅 주먹밥 지푸라기 섞이 야채
아니 항상 이런 체크 바라 단골 맛집 족발 쫄깃쫄깃 맛나 매우 불닭 맛있 전화 주문 사장 엄청 친절 맛있 냄새 맛있 매운맛
양념 너무 서비스 너무 맛있 너무 푸짐 만족 다른 지점 족발 여기 조금 이용 친절 빨리 나오 냉채 족발 사이드 메뉴 
두부찌개 시키 맛있 깔끔 족발 깻잎 그렇 밑반찬 너무 신선 재방 문의 부추 거리 수칙 근처 지키 잡내 별로 족발 보쌈 처음
남편 맛있 푸짐하 가장 맛있 족발 맛있 맛있 어제 배달 시키 아쉽 ㅠㅠ 족발 맛나 특히 맛나 ㅎㅎ 맛있 족발 맛있 들르 
가족 어느 맛나 냄새 맛있

Keywords: [('양념', 0.3666), ('막국수', 0.3208), ('족발', 0.2907), ('보쌈', 0.2567), ('야채', 0.2249), ('배추', 0.0927)]

Restaurant : 가장맛있는족발 구의역점
Menu list :['', '왕족', '알뜰', ')', '불족발', '실속', '보쌈', '싱글족발', '1', '발', '싱글보쌈', '어리굴젓', '통영', '불', '인', '하', 'g', '왕족발', '야채', '새우젓', '알', '1인', '마늘', '수제', '싱글불족발', '실속알뜰보쌈', '족발하고', '쌈야채', '양념', '막국수)', '200', '알배추', '고', '족발', '수제양념막국수', '싱글', '200g', '쌈장', '막국수', '겉절이', '배추', '쌈'] 
--------------------------------------------------------------------------------


양념 강추 ㅎㅎ 무조건 반반 ㅎㅎ 미나리 같이 오리 주물럭 아주 맛있 고기 시키 뼈탕 서비스 들깨 국물 맛있 도리 부추 
미나리 무침 편하 반반 가능 주물럭 양념 훈제 선택 가능 고기 나오 사람 가득 소주 부르 미나리무침 물론 오리 고기 맛있 
양념 모두 맛있 맛있 오리 구이 훈제 아니 오리 은근 힘들 맛있 오리 구이 서비스 알차 찌니 좋아하 오리 주물럭 정말 맛있
오리고기 유명 복불복 맛있 술집 고기 은근 동네 슴슴하 향긋 부드럽 오리 주물럭 킬링 메뉴 여기 바로 바로 무치 
미나리무침 같이 푸르 초원 활개 오리 미치 밸런스 오리고기 좋아하 느끼하 너무 맛있 마리 시키 적당 서비스 오리탕 진짜 
맛있 오리 로스 구이 마리 시키 적당 나오 오리탕 포만감 느끼 고기 양념 과하 맛있 특히 반찬 나오 미나리 향시 겉절이 
아주 맛있 내일 가족 여행 오리 주물럭 포장 주물럭 양념 반반 시키 이모 여기 오리 주물럭 미나리무침 오리고기 여기 마리
가격 비하 마리 가격 비싸 오리 주물럭 비싸 담백하 담모임도 오리로 구의동 오리 주물럭 맛집 맛있 여기 미나리 무침 오리
당연 유구 전통 구의동 유황 오리고기 찰떡 오리고기 볶음밥 오리탕 맛집 ㅎㅎ 오리 비린내 맛있 소문나 유명 해외 돌아오 
다시 설명 필요 일산 여기 오리고기 일부러 친구 만나 구의 여기 주물럭 맛있 볶음밥 찌니 좋아하 오리 주물럭 매일 
오리고기 맛있 다양 반찬 나오 가족 식사 장소 오리 주물럭 신선 미나리 무침 맛있 오리탕 가성비 최고 강변 최고 맛집 
하나 주물럭 야무지 자칫 오리 고기 느끼 마무리 나오 시간 걸리 여기 주변 오리 고기 가깝 오리고기 맛있 80 어르신 동네 
솔직 다른 존맛이네 느끼 당연히 오리 주물럭 마리 생각 오리 구이 맛있 오리고기 맛있 미나리 무침 맛있 이곳 남녀노소 
맛있 구의역 맛집 미나리 곁들이 오리 주물럭 정말 맛있 개인 오리 맛나 한잔 기울이 건강 음식 그렇 마시 취기 오르 느낌 
진짜 반찬 하나같이 맛있 메인 주물럭 훈제 겁나마싰어요 ㅜㅠ 볶음밥 오리 맛집 강력 추천 너무 친절 오리고기 오리탕 
맛있 오리 마리 친절 오리 반찬 특히 미나리무침 서비스 너무 맛있 특별 부모 함께 오리고기 볶음밥 짱짱 진짜 오래 오리 
맛집 마리 시키 다음 마리 엄청 든든 미나리무침 맛있 오리 깔끔 가성비 괜찮 오리 주물럭 구의동 30 토박이 반찬 겁나 
맛있 여기 아줌마 아저 겁나 친절 ㅜㅜ 오리 주물럭 양념 주물 오리 주물럭 맛있 지나가 항상 사람 맛집 방문 아주머니 
너무 친절 오리고기 맛있 홀로 아주머니 진짜 대단 혼자 컨트롤 허리 한번 안쓰럽 다음 양념 묻히 주물럭 같이 나오 미나리
강하 볶음밥 맛있 오리 주물럭 양념 맛있 구의역 오리 주물럭 매콤하 ㅋㅋㅋ 고기 먹으 나오 자리 뼈탕 맛있 양념 주물럭 
정말 맛있 마지막 볶음밥 너무 가격 대비 너무 고기 서비스 나오 오리 주물럭 오리뼈탕 무료 오랜 단골

Keywords: [('오리', 0.5379), ('양념', 0.3522), ('훈제', 0.2934), ('마리', 0.2768), ('뼈탕', 0.2058), ('주물럭', 0.1575)]

Restaurant : 강나루 유황오리주물럭 본점
Menu list :['탕', '양념', '양념주물럭', '오리주물럭', '오리', '주물럭', '마리', '모듬', '훈제', '뼈탕', '한', '모듬한마리', '뼈', '훈제오리'] 
--------------------------------------------------------------------------------


구의역 고기 토시 갈비 섯구이 구성 소고기 세트 주문 450 인분 고기 직접 고기 정말 맛있 사이드 주문 쫄면 된장찌개 맛있
반찬 정갈하 양념 다양 맛보 재미 특히 정말 고기 나오 냄새 장점 데이트 장소 회식 장소 저녁 시간 걸리 웨이팅 오래 직원
세상 친절 너무 맛있 식사 사장 최고 고기 정갈 나오 다양 고기 같이 여기 파김치 아주 요물 항상 방문 돼지고기 소고기 
너무 웨이팅 때문 그렇 서비스 감동 웨이팅 때문 힘들 그렇 꾸준히 방문 오랜만 정말 괜찮 고깃집 개인 보다 맛있 소고기 
세트 토시 갈비 나오 토시 대박 드시 밑반찬 나오 특히 생채 최고 고기 직접 시스템 체계 부모 모시 방문 구의역 고기집 
소고기 일단 테이블 옷장 소고기 set 주문 토시 먼저 정성 스럽 다음 촉촉 송이버섯 마지막 부드럽 갈비 정말 든든 소금 
와사비 맛있 여기 무생 무침 진짜 맛있 마지막 식사 메뉴 김치찌개 야무지 여러분 고공 ㅋㅋ 오랜만 맛있 고깃집 돼지고기 
맛있 달라 삼겹살 납득 가능 반찬 종류 다양 부위 방법 다르 추천 양념 다르 맛있 김치찌개 진하 고기 듬뿍 직원 친절 친구
가족 구의역 고기 맛집 구의동 삼겹살 맛집 구의동 소고기 맛집 발견 고공 소고기 세트 주문 토시 엄청 맛있 고기 직접 
편하 사이드 메뉴 된장찌개 맛있 다음 방문 방문 의사 500 소고기 세트 너무 맛있 직접 맛있 일단 밑반찬 너무 맛있 옥수수
가루 처음 이것 진짜 맛있 방문 예정 구의역 출구 인근 여러 가게 이전 자양동 들르 프리미엄 숙성 전문점 들어오 깔끔 
널찍 직원 친절 인상 구의역 고기집 숙성 돼지고기 숙성 소고기 전문 구의동 고깃집 직원 고기 가족 어른 들르 보이 자체 
훌륭 숙성 돈육 여러 소금 소스 제공 고기 제대로 즐기 광진 구청 고기집 들르 즐기 기다리 우와 가족 모임 친척 언니 맛있
예약 같이 방문 즈으으응말 징쨔 맛있 ㅠㅠ 고기 계속 무엇 옥수수 염료 진짜 색다르 고소하 맛있 직원 친절 물어보 설명 
너무 소스류 다양 너무 맛있 고이 너무 만족 반해쓰요 ㅠㅠ 구의역 고깃집 소고기 세트 가격 괜찮 괜찮 가격 양념장 여러 
취향 개인 사물함 데이트 구의역 골목 고깃집 고공 고기 가성비 즐기 고공 제일 맛있 고기 고기 ㅎㅎ 전문가 제대로 고기 
진짜 너무 너무 맛있 직원 친절 옷장 생각 직원 친절 대화 집중 고기 맛있 세트메뉴 주문 가성비 괜찮 하우스 인도 와인 
한잔 고기 맛있 곁들이 정갈 깔끔 맛있 너무 테이블 별로 캐비넷 정말 환경 연료 옥수수 사용 인상깊 부모 함께 고기 맛있 
쾌적 반드시 구의역 가깝 소고기 돼지고기 맛집 직접 친절 고기 설명 맛있 방법 알리 특이 옥수수 연료 곁들이 소스 다양 
조합 재미 보관 따로 고기 냄새 배이 걱정 노노 고기 인정 고기 진짜 두툼하 맛있 진짜 도리 기본 정갈하 김치 하나하나 
맛있 친구 다음 매꼼쫄 맛있 생각나 쫄면 후식 맛집 자리 단체 보관 캐비닛 넉넉 보관 회식 구의역 고기집 강추 소고기 
고공 다녀오 소고기 느끼 직접 편하 식사 고기 고공 고기 너무 편하 글라스 와인 한잔 마시 매장 단체 고기 진심 직원 
열심히 밑반찬 소스 진짜 다양 외투 보관 따로 매우 편리 화장실 가게 내부 나가 필요 돼지고기 소고기 못지않 맛있 무엇 
직원 너무 친절 기억 진짜 인생 고기집 돼지 세트 시키 진짜 인생 고기 만나 목살 태어나 목살 맛있 가게 직원 엄청 친절 
옥수수 연료 사용 고기 원리 너무 신기 방문 차분 분위기 조명 은은하 대화 고기 없이 살살 기본 제공 밑반찬 유자 새우젓 
옥수수 빵가루 소고기 찍어먹으먄 딘쨔 맛탱구리 구의동 고기 고공 숙성 고기 부드럽 반하 버리 매장 깨끗 친절 직원 직접 
데이트 모임 회식 구성 가격 괜찮 사이드 반찬 찌개 맛도리예요 하우스 와인 5000 소고기 넘넘 어울리 함께 드시 가성비 
겁나 너무 친절 피크 타임 무조건 웨이팅 참고 _ㅠ 그대로 기다리 맛집 데이트 방문 구의역 자주 인테리어 너무 깔끔 고기 
퀄리티 너무 처음 직접 너무 편하 옷장 고깃집 대하 배려 음식 퀄리티 소중 사람 방문 너무 구의역 고기 생각나 고공 강추 
사장 여기 고기 월급 청결 고기 신선하 다양 양념 모든 너무 친절 고기 너무 맛있 이런 이제 ㅎㅎ 기본 반찬 디테일 신경 
느끼 번창 ㅎㅎ 구의동 고기 데려가 고기집 반찬 다양 와사비 파절 파김치 빵가루 고추 새우 마늘 생채 등등 고기 다양 
사이드 메뉴 매콤 주문 필수 완전 프레시 입가심 고기 같이 돼지 소고기 세트 합리 가격 종류 나오 직원 직접 맛있 한몫 
자주 방문 고기집 목살 가브리살 오겹살 직접 너무 편하 반찬 종류 엄청 다양 고기 부위 추천 소스 가브리 엄청 부드럽 
목살 너무 맛있 된장찌개 고기 어울리 사장 친절 여기 고기 완전 대박 무지 밑반찬 소스 종류 다양 재미 번창 바라 직접 
맛있 구의 맛집 고공 소고기 세트 토시 갈비 버섯 구이 포함 고기 퀄리티 자체 엄청 깔끔 보관 마련 나오 냄새 하나 완벽 
고기집 동네 여러 방문 부르 고공 고기 직접 소스 다양 다양 밑반찬 맛있 고기 맛있 쫄면 완전 강추 구의역 근처 고기집 
부드럽 야들야들 식감 일품 갈비 토시 맛있 근처 위치 접근 가족 모임 데이트 회식 다양 목적 방문 구의동 삼겹살 목살 
맛집 고공 컨디션 고기 가지 사용 퀄리티 자부심 가지 고기집 상응 부드럽 식감 터지 육즙 하모니 매우 훌륭 밑반찬 맛있 
고기 어울리 무엇 직원 모두 친절 배부르 행복 식사 직원 친절 테이블 겉옷 소지품 보관 옷장 따로 고기 가격 대비 훌륭 
직원 모임 너무 편하 소고기 완전 ㅎㅅㅎ 소고기 세트 적당하 맛있 깔끔 회식 모임 장소 제격 고기 직접 직원 친절 전체 
분위기 정말 반찬 고기 맛있 특히 오겹살 너무 맛있 고공 소스 곁들이 완전 고공 추천 올해 고기 최고 살살 고기 토시 갈비
맛있 남친 대화 없이 흡입 버리 매장 분위기 특별 소금 반찬 고기 조합 최고 구의역 최고 소고기 맛집 인졍 정말 강추 고기
서비스 넘넘 일단 고급 스럽 분위기 윤기 좌르르 마블링 넘치 고기 제공 고소 내리 느낌 고기 더불 소스 방법 친절 설명 
고기 접시 올리 서비스 넘넘 최고 반찬 직접 개발 레시피 만드 정성 가득 특히 유자 고추 소스 고기 정말 최고 가격 비싸 
단체 회식 강추 이제서야 고깃집 저장 너무 맛있 직접 다양 소스 곁들이 맛있 설명 당연 너무 맛있 특히 목살 너무 부드럽 
육즙 팡팡 터지 황홀했습니댜 자주 방문 깔끔 밑반찬 맛있 직원 친절 옥수수 연료 독특 소고기 부드럽 육즙 가득 맛있 완전
추천 쫄면 맛있 맛깔 부모 모시 존맛탱 소고기 살살 돼지 목살 오겹살 미치 매장 깔끔 고기 편하 맛있 테이블 캐비넷 냄새 
보관 숙성 소고기 그렇 육즙 풍성 곁들이 반찬 하나 같이 맛있 고기 맛있 고기 알맞 최고 가격 대비 기질 미치 고기 두툼 
너무 너무 맛있 가격 확인 ㅋㅋㅋㅋ 너무 만족 스럽 부모 모시 다시 분위기 연말 분위기 신경 찾아가 그만큼 보람 직접 
알리 오마카세 느낌 구의역 근처 맛집 인정 소고기 부드럽 육즙 맛있 옥수수 대신 고기 고소 달달하 더욱 맛있 느끼 직원 
친절 직접 엄청 편하 김치찌개 너무 맛있 고기 빛깔 보이 미치 너무 맛있 고기 퀄리티 무엇 직접 편안히 반찬 소스 정말 
완벽 돼지 세트 구성 엄청 두껍 맛있 사장 친절 고기 너무 토시 갈비 맛있 밑반찬 정갈하 너무 특별 방문 다음 돼지고기 
소고기 직접 너무 맛나 디저트 된찌밥 추천 오늘 의역 고기 고공 고깃집 방문 소고기 세트 돼지 세트 다양 부위 즐기 위하 
시키 고기 직원 직접 편하 고기 질감 너무 사르르 느끼하 정도 기름 이미 황홀 상태 같이 청양고추 소스 마늘 소스 와사비 
조합 매우 엄청 느기 소고기 기름 고기 매우 조합 특히 여기 소금 종류 고기 부위 소금 입맛 조합 개인 입맛 다음 방문 
직접 괜찮 소고기 고기 직접 육즙 좔좔 너무 맛있 맛있 절로 나오 쫄면 베스트 메뉴 시키 쫄깃하 식감 너무 맛있 사장 친절
음식 깔끔 너무 맛있 가족 함께 방문 고기 맛있 반찬 가족 연인 친구 분위기 고기집 핫플 생기 이유 평소 별로 사진 사진 
까먹 버리 저희 부부 학생 특별 방문 고깃집 이제 아기 가지 이번 처음 찾아가 역시 고공 정말 맛있 고기 맛있 동안 많이 
챙기 정말 감사 이미 여러 차례 방문 역시 이전 회식 고기 방문 가

Keywords: [('소고기', 0.5594), ('쫄면', 0.5199), ('돼지', 0.5149), ('오겹살', 0.464), ('토시', 0.4314), ('매콤', 0.4055), ('고공', 0.399), ('목살', 0.3876), ('갈비', 0.3875), ('가브리', 0.3594), ('김치찌개', 0.3501), ('가브리살', 0.3236), ('세트', 0.2768), ('된장찌개', 0.2096)]

Restaurant : 고공 구의점
Menu list :['', '돼지세트', '쫄면', '목살', '고공', '김치찌개', '갈비살', '밥', '된장찌개', '겹', '갈비', '세트', '오겹살', '소고기', '고공세트', '토시살', '살', '면', '오', '매콤', '돼지', '소고기세트', '토시', '가브리살', '쫄', '가브리'] 
--------------------------------------------------------------------------------


보글보글 김치찌개 두부 추가 라면 사리 필수 김치찌개 좋아하 역시 양도 푸짐하 김치 특히 넘넘 맛나 이사 가족 간단 식사
김치찌개 완전 맛나 김치찌개 김치 계란말이 맛나 김치찌개 완전 강추 이제 자주 방문 포장 가성비 최고 고기 김치찌개 
아차산 ㅎㅎ 대박 아차산역 이렇 맛집 숨기 가격 이양 기본 반찬 훌륭 포장 혼밥 이용 아차산 등산 김치 진짜 맛있 김치 
고기 토치 한번 두부 김치 가득 계란말이 두툼 부드럽 밑반찬 맛있 점심 메뉴 강추 아차산역 출구 아주 가깝 김치 국물 
슥슥 비비 환상 계란말이 두껍 퐁신해 세트 주문 귀엽 계란 후라이 올라가 맛나 ㅎㅎ 자리 화장실 내부 깨끗 단체 모임 
등산 끝나 따뜻 부모 어린이대공원 후문 아차산역 인근 들르 칼칼하 한식 김치찌개 발걸음 주문 경우 고기 푸짐 김치 000 
즐기 탁월 가성비 아차산역 밥집 거기 셀프 반찬 소시지 케첩 볶음 마련 혼밥 보이 맛있 정말 푸짐 셀프 반찬 맛있 음식 
달달하 매콤하 맵찔이 가격 이런 퀄리티 너무 오랜만 아차산 등산 혼밥 모임 만족 스럽 김찌 맵기 조절 가능 칼칼하 제육 
짭졸 계란말이 주문 불구 도톰 부들 김찌와계 찰떡궁합 사장 이세용 아차산역 김치 끝판왕 고기 김치 김치 아차산 본점 
엄청 구수하 반찬 무한 리필 김치 인분 다른 인분 족히 고기 엄청 많이 아차산 등반 많이 김치 끊임없이 중독 오늘 기본 
고기 시키 김치 육수 조화 못하 아쉽 밑반찬 매장 만드 반찬 아니 그릇 수저 불편 ㅜㅜ 고기 김치찌개 주문 고기 진짜 너무
많이 터지 게다가 김치찌개 맛있 후라이 올라가 대접 쩝쩝 박사 계란말이 실화 퐁실퐁실 진짜 부드럽 쌉꿀마 김치찌개 
계란말이 제육 가성비 넘치 흐르 아차산 밥집 고기 반하 김치찌개 드시 김치찌개 세트 메뉴 시키 장난 아니 푸짐하 치사 
무조건 추가 추천 계란말이 맛있 기본 계란 후라이 올리 너무 밑반찬 무한 셀프 아차산 등산 점심 배달 김치찌개 땡기 
아차산역 근처 지나가 자주 매번 김치 좋아하 정말 맛있 자극 재료 푸짐 들어가 두부 고기 가득 배부르 계란말이 퐁신퐁 
김치 같이 밥도둑 반찬 셀프 반찬 리필 가능 맛있 다음 아차산역 맛집 인정 아차산 맛집 하나 다녀오 사장 친절 반찬 무한 
리필 만족 스럽 아이 함께 즐기 반찬 불고기 제육 남녀노소 누구 함께 즐기 사이드 계란말이 스팸 구이 시키 엄청 부드럽 
촉촉하 터지 무조건 들리 이제 후회 여기 필수 사장 많이 진정 밥도둑 반찬 셀프바도 넉넉 리필 혼밥 마음 아차산 등산 
맛집 인정 오랜만 김치찌개 너무 방문 김치찌개 엄청 푸짐하 고기 가득 사이드 메뉴 계란말이 스팸 구이 김치찌개 같이 
너무 어울리 아차산역 이런 밥집 다음 아차산 가깝 등산 점심 저녁 부모 같이 점심 위하 방문 진짜 푸짐하 정말 김치찌개 
등등 다양 메뉴 취향 여러 가지 필요 추천 드리 식당 ㅎㅎ 생일 집밥 방문 저희 엄마 찌개 건더기 많이 스타일 김치 고기 
두부 많이 너무 아차산역 근처 맛집 방문 다양 세트 메뉴 혼밥 든든하 식사 김치 돼지고기 듬뿍 들어가 국물 풍미 특히 
라면 사리 정말 맛있 진짜 방문 뜨끈하 김치찌개 풀리 따땃 특히 김치찌개 고기 너무 맛있 다음 고기 시키 제대로 음미 
불고기 맛있 계란 쓱쓱 비비 너무 맛있 번창 김치찌개 세트 불고기 세트 시키 시키 계란말이 역시 맛있 불고기 맛나 맛있 
계란말이 서비스 반찬 무한 리필 특히 제육 기대 적당 완전 맛있 아이 함께 제육볶음 매력 흠뻑 빠지 간장 소스 고추가루 
소스 너무 맛있 저희 아들 두부 엄청 김치찌개 두부 잔뜩 무슨 고봉 등치 남기 정도 사장 인심 대단하 가족 외식 장소 우리
아이 제육볶음 고기 유명 고기 김치찌개 아차산 김치 동네 주민 이미 ㅎㅎ 고기 잡내 엄청 푸짐 적당히 칼칼하 맛나 아니 
가격 저렴 가성비 반찬 소세지 샐러드 등등 흰밥 반숙 계후 올리 시키 고기 찌개세트에 라면 사리 계란말이 술술 들어가 
기반 김치찌개 계란말이 이름 그대로 왕크네 다른 김치찌개 맛보 방문 방문 너무 맛있 뚜벅 찾아오 어렵 힘들 도보 직원 
친절 음식 양도 인분 같이 엄청 김치찌개 다시마 국물 진하 만들 아니 개인 생각 만큼 국물 진하 맛있거드 ㅋㅋㅋㅋㅋ 
조만간 날씨 풀리 올림픽 공원 와바야겠 어용 계란말이 인용 사이즈 시키 김치찌개 맛집 가득 천하 김치찌개 맛있 계란말이
최고 아차산역 맛집 인정 방문 특히 리필 제격 계란말이 진짜 맛있 추가 바라 김치찌개 맛집 라면 사리 추사 무조건 양푼 
계란 나오 푸짐하 가격 저렴하 가성비 김치 두부 고기 정말로 김치찌개 일반 주문 고기 김치찌개 고기 많이 들어가 
김치찌개 세트 돼지 제육볶음 불고기 베스트 맛집 회식 모임 데이트 추천 사리 보글보글 계란 후라이 올리 감칠맛 매장 
쾌적 깔끔 사장 친절 내내 기분 화장실 깔끔 기본 음식 맛있 매콤 정도 맵기 조절 찌개 대접 나오 그릇 가성비 맛있 
아차산역 출구 맛집 드디어 방문 소주에 김치찌개 맛있 김치찌개 세트 라면 사리 계란말이 맥주 김치찌개 진짜 맛있 기본 
김치 추가 그렇 김치 대박 다음 참치 꽁치 주문 반찬 셀프 무한 아차산역 밥집 회사원 동네 주민 식사 시간 많이 남녀노소 
불문 들어오 푸짐 가격 저렴하 ㅎㅎ 반찬 종류 다양 잘리 야무지 최고 최고 혼밥 추천 추천 방문 예정 최애 김치찌개 김치 
맛집 평생 예정 ㅎㅎㅎ 여기 김치 진짜 맛있 김치 고기 맛있 메뉴 혼밥 테이블 단체 식사 날씨 뜨끈 얼큰 김치찌개 맛있 
입맛 다시 돌아오 ㅎㅎ 신랑 아차산역 놀러오 맛집 검색 진짜 푸짐하 맛나 정말 진짜 구수 맛있 사이드 시키 고추장 
제육볶음 너무 도리 아차산 들리 맛있 아차산 혼밥 식사 안주 아이 아이 먹이 우삼겹 사이드 메뉴 먹이 괜찮 주문 다양 
혼밥 편하 드시 맛있 김치찌개 고기 스팸 참치 선택 주문 보통 고기 통일 김치찌개 좋아하 선택 직장 점심 장소 김치찌개 
선택 가능 계란말이 한국 김치찌개 고기 김치찌개 세트 배부르 세트 계란 후라이 대접 포함 가성비 진짜 느끼 김치찌개 
대박 칼칼 매콤 조금 고기 김치 거의 비율 맛나 간장 제육 맛있 포장 반찬 포함 네이버 메뉴 세트 반찬 국도 포장 제공 
아니 안내 생각 고기 맛있 포장 이용 구매 고기 엄청 자작 국물 김치 배달 종종 매장 방문 처음 매장 깔끔 놀래 아차산역 
맛집 인정 고기 김치찌개 계란 고기 반하 추천 드리 원래 김치찌개 고기 좋아하 스팸 진짜 맛있 두부사리 추가 너무 
부들부들 고소 맛있 1500 추가 세트 시키 계란 후라이 나오 김치 같이 맛있 반찬 나오 세상 꿀맛 얼른 김치 드시 비비 맛있
너무 맛있 소주 땡기 ㅋㅎㅋㅎㅋ 사리 고기 정말 김치 세트 일단 부족 만족 스럽 다음 제육 김치 찌개 추천 어느 하나 가지
아주 너무너무 맛있 고기 김치찌개 김치 아차산역 맛집 추천 드리 고기 들어가 김치찌개 너무 가성비 김치찌개 김치 
계란말이 아차산역 혼밥 김치찌개 정말 맛나 고기 가득 국물 깔끔 다음 김치 맛보 다시금 방문 예약 김치찌개 생각 무조건 
가게 리필 진짜 김치찌개 정말 맛있 ㅜㅜ 직원 항상 친절 가성비 너무 오래오래 반주 그렇 술집 분위기 아니 아이 데리 
너무 집밥 이너낌 너무 드디어 김치찌개 맛집 밥집 생기 오랜만 저녁 끝내 마시 김치찌개 맛있 ㅠㅠㅠ 다음 라면 사리 추가
김치찌개 세트 터지 방문 리뷰 처음 일단 인분 주문 가능 다양 밑반찬 매일 매일 조금 바꾸 나오 테이블 저희 테이블 
쓰레기통 여자 직원 챙기 계산 친절 남자 직원 개선 필요 보이 어린이대공원 산책 밥집 오랜만 김치찌개 여기 진심 어린이 
공원 맛집 장난 아니 조합 최고 세트 시키 계란 후라이 같이 나오 너무 계란말이 사리 같이 끝내주 다음 혼밥 방문 너무 
사진 정신 없이 챙기 아차산 밥집 김치 생기 가성비 정말 고기 많이 슬프 세트메뉴 계란 나오 육수 끓이 인생 김치 고기 
맛있 아차산 맛집 기대 모두 부드럽 두부 김치 고기 꿀맛 주변 이런 밥집 생기 너무 김치찌개 얼큰 시원하 너무 맛있 맛나 
제육 아차산역 맛집 추천 이용 아차산 맛집 고기 김치찌개 김치찌개 무한 리필 너무 다음 김치 맛있 김치찌개 기대 아차산 
근처 아차산 맛집 ㅎㅎ 새로 생기 사진 맛있 보이 들어가 밑반찬 나오 세트 시키 사진 너무 반찬 ㅜㅜ 계란 후라이 딴판 
김치 계란말이 너무 맛있 우리 동네 맛있 

Keywords: [('김치', 0.4301), ('제육', 0.3079), ('김치찌개', 0.3019), ('세트', 0.2765), ('고추장', 0.212), ('간장', 0.1731), ('불고기', 0.0878), ('고기', 0.0389)]

Restaurant : 고기반햄반김치찌개&김치찜 아차산본점
Menu list :['4인', '김치찌개', '불고기', '1', '반', '간장제육', '제육', '인', '세트', '고기반', '2', '1인', '2인', '3', '말', '소불고기', '김치찜', '란', '왕', '간장', '소', '왕계란말이', '이', '3인', '고기', '고추장제육', '계', '4', '김치', '찜', '고추장'] 
--------------------------------------------------------------------------------


조개 무침 벌교 꼬막 대구탕 새조개 무침 미치 조개 진짜 비싸 많이 헿헿 꼬막 기준 살짝 벌교 꼬막 38000 인분 정도 
하나하나 실하 부드럽 양념 없이 그냥 가위 까먹 재미 생굴회가 마트 신선 맛있 동태 인분 푸짐하 쭈꾸미 너무 싱싱하 맛있
무침 30 000 재미 무침 양념 무침 우리 입맛 역시 초장 제맛 고향 모든 메뉴 프로젝트 선택 18000 바다 내음 다음 무침 
상큼 음식 냉면 대신 이름 생소 장뚱어 특이 아니 추어탕 깔끔 음식 음식 요즘 제철 함께 싱싱 맛있 재미 무침 맛있 쭈꾸미
구이 맛있 볶음밥 맛있 사실 밑반찬 뜸하 여름 땡기 쭈꾸미 구이 볶음밥 짱뚱어탕 완전 어울리 세트 여름 역시 최고 근대 
화장실 타일 너무 미끄럽 엉덩방아 고생 좋아 쭈꾸미 포장 비추 반찬 콩나물 재미 무침 재료 신선하 싱그럽 맛있 물회 맛있
쭈꾸미 구이 시그니처 맛때매 지인 소개 방문 쭈꾸미 싱싱 양념 강하 맛있 짱뚱어탕도 별미 다음 방문 ㅎㅎ 쭈꾸미 볶음 
짱뚱어탕 밑반찬 맛있 대구탕 찌개 대구 부드럽 국물 깔끔 맛있 쭈꾸미 탱글탱글 맛있 주꾸미 볶음 매콤하 맛있 반찬 입맛 
재미 무침 신선 향긋하 미나리 찰떡궁합 사장 쭈꾸미랑 대구탕 맛있 쭈꾸미 구이 나오 반찬 맛나 양념 꼬막 맛나 사진 아쉽
쭈꾸미 맛있 특히 오이 무침 김치 구옥 개조 고향 방문 느낌 쭈꾸미 양념 흔하 슴슴하 자극 진짜 많이 단골집 쭈꾸미 조개 
샤브샤브 가격 비싸 조개 샤브샤브 맛있 꼬막 맛있 쭈꾸미 맛있 신선하 쭈꾸미 최고 짱뚱어탕 순천만 그대로 대구탕 정말 
맛있 밑반찬 맛있 간재미회 무침 특히 강추 나가 입맛 돌아오 쭈꾸미 반찬 맛있 쭈꾸미 구이 처음 쭈꾸미 볶음 언제 맛나 
친절 깨끗하 맛있 사진 짱뚱어탕 맛있 재미 무침 별미 쭈꾸미 짱뚱어탕 시원하 바다 코로나 19 여름 이렇게 달래 쭈꾸미 
구이 서울 만나 어렵 짱뚱어 맛나 반찬 기본 이상 짱뚱어탕 ㅋㅋ 쭈꾸미 가성비 최악 물회 최악 물회 맛있 재미 무침 맛있 
구미 볶음 짱뚱어 별로 새조개샤브 짱짱 비싸 쭈꾸미 담백하 밑반찬 겨울 조개 샤브샤브 맛있 계절 메뉴 회랑 겨울 무침 
정말 맛있 밑반찬 맛있 회랑 쭈꾸미 최고 마당 주차 가능 짱뚱어탕 부모 모시 다녀오 추어탕 비슷 담백 구의동 맛집 강추 
쭈꾸미 볶음밥 마무리 짱뚱어 반찬 너무 맛있 짱뚱어탕 쭈꾸미 맛있 물건 사용 생물 쭈꾸미 양념 허영만 백반 기행 나오 
웨이팅 낙지 꼬막 맛있 쭈꾸미 구이 쭈꾸미랑 짱뚱 맛있

Keywords: [('짱뚱어탕', 0.5677), ('구이', 0.5641), ('대구', 0.4723), ('쭈꾸미', 0.436), ('양념', 0.4096), ('꼬막', 0.3736), ('재미', 0.3569), ('찌개', 0.3097), ('무침', 0.2758), ('물회', 0.273), ('벌교', 0.1928)]

Restaurant : 고향집
Menu list :['회', '구이', '새꼬막', '가자미', '참꼬막양념', '생태찌개', '꼬막', '간장게장', '재미', '간', '새', '무침', '게장', '굴', '가자미무침', '생태', '간재미', '회무침', '참', '쭈꾸미구이', '물회', '양념', '생', '짱뚱어탕', '새꼬막양념', '간장', '벌교', '생대구찌개', '찌개', '대구', '쭈꾸미', '물', '벌교참꼬막', '굴회무침'] 
--------------------------------------------------------------------------------
